In [18]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import cv2
import numpy as np


In [19]:
import glob
vehicle_images=glob.glob("dataset/vehicles/*/*.png")
non_vehicle_images=glob.glob("dataset/non-vehicles/*/*.png")
print("Dataset Contains: {} Vehicle and {} Non-Vehicle images".format(len(vehicle_images),len(non_vehicle_images)))

Dataset Contains: 8792 Vehicle and 8968 Non-Vehicle images


In [20]:
from skimage.feature import hog
def get_features(img,cell_per_block=2,pix_per_cell=8,orient=9, color_hist_bins=32, img_resize=(64,64)):
#     returns the feature vector for the image
    if(img_resize != (img.shape[0]),img.shape[1]):
        img=cv2.resize(img,img_resize)
#     get hog features before colorspace conversion
    feat=hog(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY), orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
#     Convert to required colorspace
    img=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    for i in range(3):
#         loop over three channels
        ch_hist = np.histogram(img[:,:,i],bins=color_hist_bins,range=(0,256))[0]
        feat = np.concatenate((feat,ch_hist))
    return feat

Read the images and create X_raw, y_raw


In [21]:
vehicle_features=[]
for vehicle_image in vehicle_images:
    vehicle_features.append(get_features(cv2.imread(vehicle_image)))
non_vehicle_features=[]
for non_vehicle_image in non_vehicle_images:
    non_vehicle_features.append(get_features(cv2.imread(non_vehicle_image)))


Scale the X features

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

X_raw = np.vstack((vehicle_features,non_vehicle_features)).astype(np.float64)
y_raw = np.hstack((np.ones(len(vehicle_features)),np.zeros(len(non_vehicle_features))))

stdscaler = StandardScaler().fit(X_raw)
X_raw_scaled = stdscaler.transform(X_raw)

# Test train split
X_train, X_test, y_train, y_test = train_test_split(
    X_raw_scaled, y_raw, test_size=0.2, random_state=12)

In [23]:
# Size of dataset
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (14208, 1860)
X_test shape: (3552, 1860)
y_train shape: (14208,)
y_test shape: (3552,)


SVM GridSearch CV

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
svm_clf=svm.SVC(kernel='rbf',C=10,gamma=0.001)

Fit SVM calassifier

In [25]:
svm_clf.fit(X_train,y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Save the model to re-use (model persistance)

In [26]:
from sklearn.externals import joblib

joblib.dump(svm_clf,'model_svm3.pkl')
joblib.dump(stdscaler, 'stdscaler3.pkl')
# Save the data
joblib.dump(np.array([X_train, X_test, y_train, y_test]),'dataset3_bkp.pkl')

['dataset3_bkp.pkl']

In [27]:
# Print Test accuract:
print(svm_clf.score(X_test,y_test))

0.989301801802
